### Задача

Основываясь на пробах нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль.

### План и описание работы

Стандартные шаги для выбора локации:
- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

###  Условия задачи
 - Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
 - При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
 - Бюджет на разработку скважин в регионе — 10 млрд рублей.
 - При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
 - После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.<br>

**Содержание колонок:**
 - id — уникальный идентификатор скважины;
 - f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
 - product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
state = np.random.RandomState(12345)
region_1 = pd.read_csv('/datasets/geo_data_0.csv')
region_2 = pd.read_csv('/datasets/geo_data_1.csv')
region_3 = pd.read_csv('/datasets/geo_data_2.csv')

## Обучение и проверка модели

Обучим и проверим модель для каждого региона<br>
 - 2.1. Разобьем данные на обучающую и валидационную выборки в соотношении 75:25.

In [2]:
target_region_1 = region_1['product']
features_region_1 = region_1.drop(['product','id'],axis=1)

target_region_2 = region_2['product']
features_region_2 = region_2.drop(['product','id'],axis=1)

target_region_3 = region_3['product']
features_region_3 = region_3.drop(['product','id'],axis=1)

features_train_region_1,features_valid_region_1,target_train_region_1,target_valid_region_1 = \
        train_test_split(features_region_1,target_region_1,test_size=0.25,random_state = state)

features_train_region_2,features_valid_region_2,target_train_region_2,target_valid_region_2 = \
        train_test_split(features_region_2,target_region_2,test_size=0.25,random_state = state)

features_train_region_3,features_valid_region_3,target_train_region_3,target_valid_region_3 = \
        train_test_split(features_region_3,target_region_3,test_size=0.25,random_state = state)

 - 2.2. Обучим модель и сделаем предсказания на валидационной выборке.
 - 2.3. Сохраним предсказания и правильные ответы на валидационной выборке.

In [3]:
model = LinearRegression()

model.fit(features_train_region_1,target_train_region_1)
prediction_model_1 = model.predict(features_valid_region_1)
prediction_model_1 = pd.Series(prediction_model_1, index = features_valid_region_1.index)

model.fit(features_train_region_2,target_train_region_2)
prediction_model_2 = model.predict(features_valid_region_2)
prediction_model_2 = pd.Series(prediction_model_2, index = features_valid_region_2.index)

model.fit(features_train_region_3,target_train_region_3)
prediction_model_3 = model.predict(features_valid_region_3)
prediction_model_3 = pd.Series(prediction_model_3, index = features_valid_region_3.index)

 - 2.4. Напечатаем на экране средний запас предсказанного сырья и RMSE модели.

In [4]:
print('Средний запас предсказанного сырья по региону 1 (тыс.бареллей):   ',prediction_model_1.mean())
print('RMSE модели на данных по региону 1:   ',mean_squared_error(prediction_model_1,target_valid_region_1)**0.5)
print()
print('Средний запас предсказанного сырья по региону 2 (тыс.бареллей):   ',prediction_model_2.mean())
print('RMSE модели на данных по региону 2:   ',mean_squared_error(prediction_model_2,target_valid_region_2)**0.5)
print()
print('Средний запас предсказанного сырья по региону 3 (тыс.бареллей):   ',prediction_model_3.mean())
print('RMSE модели на данных по региону 3:   ',mean_squared_error(prediction_model_3,target_valid_region_3)**0.5)

Средний запас предсказанного сырья по региону 1 (тыс.бареллей):    92.59256778438038
RMSE модели на данных по региону 1:    37.5794217150813

Средний запас предсказанного сырья по региону 2 (тыс.бареллей):    68.76995145799754
RMSE модели на данных по региону 2:    0.889736773768064

Средний запас предсказанного сырья по региону 3 (тыс.бареллей):    95.087528122523
RMSE модели на данных по региону 3:    39.958042459521614


- 2.5. Проанализируем результаты.

>Вывод: У рассматриваемых регионов 1 и 3, наибольшее среднее количество бареллей, однако и РМСЕ метрика 
(среднеквадратичная ошибка предсказаний от факта) у них очень высокие 37 и 39 тыс. бареллей. В данных по региону №2 - 
среднее количество бареллей всего 68 тыс, но и ошибка в предсказании модели там относительно мала - 0.88 тыс. бареллей

## Подготовка к расчёту прибыли

Подготовимся к расчёту прибыли:
 - 3.1. Все ключевые значения для расчётов сохраним в отдельных переменных.

In [5]:
#Бюджет на разработку скважин в регионе — 10 млрд рублей.
budget = 10000000 #тысяч рублей
#При нынешних ценах доход с каждой единицы продукта составляет 450 тыс. рублей
product_price = 450 #тысяч рублей
n_try = 1000

 - 3.2. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

In [6]:
#Достаточный объём сырья для безубыточной разработки новой скважины.
#Зная что бюджеть 10 миллиардов на регион, а скважин на регион бурится 200, бюджет на каждую:
budget_one = budget/200
profit_one = budget_one/product_price
print('одна скважина для безубыточной разработки должна приносить (ед. сырья): ',profit_one)

одна скважина для безубыточной разработки должна приносить (ед. сырья):  111.11111111111111


Сравним полученный объём сырья со средним запасом в каждом регионе.

In [7]:
print('средний запас в скважине в регионе 1 (ед. сырья):',target_region_1.mean())
print('средний запас в скважине в регионе 2 (ед. сырья):',target_region_2.mean())
print('средний запас в скважине в регионе 3 (ед. сырья):',target_region_3.mean())

средний запас в скважине в регионе 1 (ед. сырья): 92.50000000000001
средний запас в скважине в регионе 2 (ед. сырья): 68.82500000000002
средний запас в скважине в регионе 3 (ед. сырья): 95.00000000000004


Вывод:средний запас сырья по разработкам в регионах, меньше чем минимально безубыточный показатель. К выбору точек добычи нужно подходить ответственно, чтобы получить прибыль а не убыток

## Расчёт прибыли и рисков 

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
 - 4.1. Выберем скважины с максимальными значениями предсказаний.
 - 4.2. Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
 - 4.3. Рассчитаем прибыль для полученного объёма сырья.
 
Посчитаем риски и прибыль для каждого региона:
 - 5.1. Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
 - 5.2. Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
 - 5.3. Предложим регион для разработки скважин.

In [8]:
def profit(target,predictions):
    count = []
    fuckup = 0
    for i in range(n_try):
        predictions_500 = predictions.sample(n=500,random_state = state, replace = True)
        pred_sorted = predictions_500.sort_values(ascending=False)
        select_points = target[pred_sorted.index][:200]
        product = select_points.sum()
        rev = product * product_price
        itog = rev - budget
        count.append(itog)
        if itog < 0:
            fuckup +=1
    count = pd.Series(count)
    return 'cредняя прогнозируемая выручка по региону (тыс.р.)',count.mean(),\
           'Доверительный интервал 95%, от',count.quantile(0.025),'до',count.quantile(0.975),\
           'Вероятность убытка {:.2%}'.format(fuckup/n_try)

print('Регион 1:',profit(target_valid_region_1,prediction_model_1))
print()
print('Регион 2:',profit(target_valid_region_2,prediction_model_2))
print()
print('Регион 3:',profit(target_valid_region_3,prediction_model_3))

Регион 1: ('cредняя прогнозируемая выручка по региону (тыс.р.)', 394284.41134055064, 'Доверительный интервал 95%, от', -69450.56685672095, 'до', 915496.1658311522, 'Вероятность убытка 6.10%')

Регион 2: ('cредняя прогнозируемая выручка по региону (тыс.р.)', 454736.3590973579, 'Доверительный интервал 95%, от', 61071.81129769981, 'до', 855914.1491428369, 'Вероятность убытка 0.70%')

Регион 3: ('cредняя прогнозируемая выручка по региону (тыс.р.)', 353664.0975248346, 'Доверительный интервал 95%, от', -162650.94694229303, 'до', 847761.8445147676, 'Вероятность убытка 7.60%')


## Вывод

Для разработки подходит регион №2, Не смотря на изначально меньший прогнозируемый моделью запас сырья в регионе (тогда было предсказания по 25000 точкам),у него наименьший риск убытка и наибольшая прогнозируемая прибыль (если отбирать 200 лучших точек из рандомных 500)